In [1]:
cd "./diff_class(class vector no-fixed)/dog2car/"

/Users/xuejiaxin/Dropbox/My Mac (Jiaxin的MacBook Pro)/Documents/GitHub/latent/pytorch-pretrained-BigGAN/diff_class(class vector no-fixed)/dog2car


In [2]:
# target image
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, display_in_terminal, one_hot_from_int)
import logging


logging.basicConfig(level=logging.INFO)

model = BigGAN.from_pretrained('biggan-deep-128')

truncation = 0.5
#class_vector = one_hot_from_names(["airliner"], batch_size=1)
class_vector = one_hot_from_int(581)
noise_vector = truncated_noise_sample(truncation=truncation, batch_size=1, seed = 90)

noise_vector = torch.from_numpy(noise_vector)
class_vector = torch.from_numpy(class_vector)


with torch.no_grad():
    output = model(noise_vector, class_vector, truncation)

save_as_images(output, "car_target")

INFO:pytorch_pretrained_biggan.model:loading model biggan-deep-128 from cache at /Users/xuejiaxin/.pytorch_pretrained_biggan/6371c3777477e4e75187da1b9b526561aac3134f38c7299a3438009ae560e20d.3434ebdfa74a8c17e0e56061cfd905fa163c92f110e88df77b47da6ce9910b48
INFO:pytorch_pretrained_biggan.model:Model config {
  "attention_layer_position": 8,
  "channel_width": 128,
  "class_embed_dim": 128,
  "eps": 0.0001,
  "layers": [
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      16
    ],
    [
      false,
      16,
      16
    ],
    [
      true,
      16,
      8
    ],
    [
      false,
      8,
      8
    ],
    [
      true,
      8,
      4
    ],
    [
      false,
      4,
      4
    ],
    [
      true,
      4,
      2
    ],
    [
      false,
      2,
      2
    ],
    [
      true,
      2,
      1
    ]
  ],
  "n_stats": 51,
  "num_classes": 1000,
  "output_dim": 128,
  "z_dim": 128
}

INFO:pytorch_pretrained_biggan.utils:Saving image to car_target_

In [3]:
from torchvision.models import squeezenet1_0
from tqdm import trange
from torch import nn

DEVICE = 'cpu'
model = model.eval().to(DEVICE)


semantic_model = squeezenet1_0(pretrained=True).to(DEVICE)
semantic_model.classifier = torch.nn.Sequential(
    torch.nn.Flatten()
    )
semantic_model = semantic_model.eval()

soft_max = nn.Softmax(dim = 1)
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)

truncation = 0.5
noise = truncated_noise_sample(truncation=truncation, batch_size=1, seed=10)
noise = torch.nn.Parameter(torch.tensor(noise, requires_grad=True).float().to(DEVICE))

#class_vector = one_hot_from_int(333)
class_vector = one_hot_from_names(['dog'], batch_size=1)
class_vector = torch.nn.Parameter(torch.tensor(class_vector, requires_grad=True).float().to(DEVICE))

optim = torch.optim.Adam([noise, class_vector], lr=0.05)

save_as_images(model(noise, class_vector, truncation), f"start")
save_as_images(output, "target")

L = []
L_pixel = []
L_semantic = []

for iteration in trange(0, 501):
    optim.zero_grad()

    #output_1 = torch.nn.functional.gumbel_softmax(class_vector, tau=1, hard=True)
    output_1 = soft_max(class_vector)

    y_hat = model(noise, output_1, truncation)

    #semantic_loss = -((semantic_model(y_hat) - semantic_model(output)) ** 2).mean() ** .5 
    semantic_loss = -cos_sim(semantic_model(y_hat), semantic_model(output))
    L_semantic.append(semantic_loss.item())

    pixel_loss = abs(y_hat - output).mean()
    L_pixel.append(pixel_loss.item())

    loss = 100*semantic_loss + 30 * pixel_loss
    L.append(loss.item())

    loss.backward()
    optim.step()


    if iteration % 1 == 0:
        print(output_1.argmax())
        save_as_images(y_hat, f"car2dog_{iteration}")

INFO:pytorch_pretrained_biggan.utils:Saving image to start_0.png
INFO:pytorch_pretrained_biggan.utils:Saving image to target_0.png
  0%|          | 1/501 [00:03<26:44,  3.21s/it]tensor(253)
INFO:pytorch_pretrained_biggan.utils:Saving image to car2dog_1_0.png
  0%|          | 2/501 [00:05<25:14,  3.04s/it]tensor(253)
INFO:pytorch_pretrained_biggan.utils:Saving image to car2dog_2_0.png
  1%|          | 3/501 [00:08<23:56,  2.89s/it]tensor(253)
INFO:pytorch_pretrained_biggan.utils:Saving image to car2dog_3_0.png
  1%|          | 4/501 [00:11<25:26,  3.07s/it]tensor(253)
INFO:pytorch_pretrained_biggan.utils:Saving image to car2dog_4_0.png
  1%|          | 5/501 [00:15<27:01,  3.27s/it]tensor(253)
INFO:pytorch_pretrained_biggan.utils:Saving image to car2dog_5_0.png
  1%|          | 6/501 [00:19<27:40,  3.35s/it]tensor(253)
INFO:pytorch_pretrained_biggan.utils:Saving image to car2dog_6_0.png
  1%|▏         | 7/501 [00:23<30:03,  3.65s/it]tensor(253)
INFO:pytorch_pretrained_biggan.utils:Savin